# Prompt Engineering Techniques


Prompt engineering is the process of designing and refining prompts for large language models (LLMs) to elicit desired responses. Different techniques exist to improve the model's performance and controllability.

In [1]:
from openai import AzureOpenAI
client = AzureOpenAI()

In [2]:
# creating a function to get outcome
def generate_response(prompt,model='gpt-4o-mini',temperature=0):
  messages = [{"role":"user","content":prompt}]
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature=temperature,
      #top_p=0.99
  )
  return response.choices[0].message.content


In [3]:
generate_response("Hi! How are you.")

"Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"


## Zero-Shot Prompting
**Description:** In zero-shot prompting, the model is given a prompt with a task and no examples of how to perform it. The model is expected to perform the task based solely on its pre-training.

**Application Areas:** Simple tasks where the model has a strong understanding from its training data, such as classification, translation, or summarization for common domains.

**When to Use:**
- When the task is straightforward and aligns well with the model's pre-training.
- When collecting examples for few-shot prompting is difficult or time-consuming.
- As a baseline to see how well the model performs without explicit examples.

**Example:**
Prompt: "Translate the following English sentence to French: 'Hello, how are you?'"



In [4]:
# zero shot prompting
prompt = f"""
Create a conversation story between child and grandparent.
"""
response = generate_response(prompt)
print(response)

**Title: The Magic of Memories**

**Setting:** A cozy living room filled with family photos, a warm fireplace, and the smell of freshly baked cookies.

---

**Child:** (snuggling into the couch) Grandma, can you tell me a story about when you were a little girl?

**Grandparent:** (smiling) Oh, I have so many stories! Which one do you want to hear? The time I got lost in the woods or the day I met my best friend?

**Child:** (eyes wide) The day you met your best friend! What happened?

**Grandparent:** (chuckling) Alright! It was a sunny day, just like today. I was playing in the park, swinging as high as I could. Suddenly, I saw a girl sitting alone on a bench, looking sad.

**Child:** What did you do?

**Grandparent:** I decided to be brave. I jumped off the swing and walked over to her. I asked, “Why are you sad?” She told me she had lost her favorite teddy bear. 

**Child:** (concerned) Oh no! Did you help her find it?

**Grandparent:** Yes! We searched everywhere. We looked behind 

In [5]:
prompt = """Classify the customer's request into one of the following categories:
- Account Balance Inquiry
- Loan Application Status
- Report Lost Card
- Fund Transfer Issue
- Credit Card Application


Customer: "Can you tell me what to do in case I misplaced my debit card?"
Category:"""
output = generate_response(prompt)
print(output)

Report Lost Card


## Few-Shot Prompting
**Description:** Few-shot prompting involves providing the model with a few examples (typically 1 to 5) of the desired input-output pairs before presenting the actual task. This helps the model understand the format and type of output expected.

**Application Areas:** Tasks where the desired output format is specific, or the task requires understanding a particular style or pattern. Examples include generating text in a specific style, answering questions in a particular format, or performing tasks on less common data.

**When to Use:**
- When zero-shot performance is not sufficient.
- When the task requires a specific output format or style.
- When the task involves domains or data less frequently seen during pre-training.
- To guide the model towards desired behavior without fine-tuning.
**Example:**
Prompt:
"Translate to Spanish:
English: 'Goodbye'
Spanish: 'Adiós'
English: 'Thank you'
Spanish: 'Gracias'
English: 'Please'
Spanish: 'Por favor'
English: 'Excuse me'
Spanish: 'Disculpe'"


In [6]:
# few shot prompting
prompt = f"""
your task is to answer in a consistent sytle:

<child>: Teach me about patience.
<grandma>: Patience is like waiting for whole day to see the moon in the evening and then sleeping after having a look at it.

<child>: Teach me about resilience.
"""
response = generate_response(prompt)
print(response)

<grandma>: Resilience is like a tree that bends in the wind but doesn’t break. It stands tall again after the storm passes, ready to grow even stronger.


#### Few shot prompting with role setup

In [9]:
result = client.chat.completions.create(
    model='gpt-4o-mini',  # e.g. gpt-35-instant
    
    temperature=0,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "That was an awesome experience"},
        {"role": "assistant", "content": "positive"},
        {"role": "user", "content": "I won't do that again"},
        {"role": "assistant", "content": "negative"},
        {"role": "user", "content": "That was not worth my time"},
        {"role": "assistant", "content": "negative"},
        {"role": "user", "content": "You can't miss this"}
    ],
)
print(result.choices[0].message.content)

positive


In [10]:
prompt = """Classify the customer's request into one of the following categories:
- Account Balance Inquiry
- Loan Application Status
- Report Lost Card
- Fund Transfer Issue
- Credit Card Application

Examples:
Customer: "What's the current balance in my savings account?"
Category: Account Balance Inquiry

Customer: "I need to know where my loan application stands."
Category: Loan Application Status

Customer: "My credit card is missing, I think I lost it."
Category: Report Lost Card

Customer: "I tried to send money to my friend but it's not going through."
Category: Fund Transfer Issue

Customer: "I want to apply for a new credit card."
Category: Credit Card Application

Customer: "Can you tell me how much money I have in my checking account?"
Category:"""

output = generate_response(prompt)
print(output)

Account Balance Inquiry


In [11]:
def get_categorization(qus):
    prompt = f"""Classify the customer's request into one of the following categories:
      - Account Balance Inquiry
      - Loan Application Status
      - Report Lost Card
      - Fund Transfer Issue
      - Credit Card Application

      Examples:
      Customer: "What's the current balance in my savings account?"
      Category: Account Balance Inquiry

      Customer: "I need to know where my loan application stands."
      Category: Loan Application Status

      Customer: "My credit card is missing, I think I lost it."
      Category: Report Lost Card

      Customer: "I tried to send money to my friend but it's not going through."
      Category: Fund Transfer Issue

      Customer: "I want to apply for a new credit card."
      Category: Credit Card Application

      Customer: {qus}
      Category:"""
    output = generate_response(prompt)
    return output


In [12]:
get_categorization("can you help me with my checkin balance?")

'Account Balance Inquiry'

In [13]:
get_categorization("I think I missed my credit card while travelling yesterday at airport, what do i do next?")

'Report Lost Card'

In [14]:
get_categorization("How do apply for a new credit card?")

'Credit Card Application'


## Chain-of-Thought (CoT) Prompting
**Description:** CoT prompting involves explicitly asking the model to show its reasoning steps or thought process before arriving at the final answer. This is often achieved by including phrases like "Let's think step by step" in the prompt. CoT can be used in a zero-shot or few-shot setting (Zero-shot CoT or Few-shot CoT).

**Application Areas:** Complex reasoning tasks such as arithmetic, logical deduction, multi-step problem-solving, and question answering that requires synthesis of information.

**When to Use:**
- When the task requires multi-step reasoning.
- When the final answer depends on intermediate steps.
- To improve the accuracy on complex problems by guiding the model through the solution process.
- To understand the model's reasoning and debug incorrect outputs.
**Example (Few-shot CoT):**
Prompt:
"Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
A: Roger started with 5 balls. He bought 2 cans with 3 balls each, so he bought 2 * 3 = 6 balls. Total balls = 5 + 6 = 11. The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 and then bought 6 more, how many apples do they have?
A: Let's think step by step. The cafeteria had 23 apples. They used 20, so they had 23 - 20 = 3 apples left. Then they bought 6 more, so they have 3 + 6 = 9 apples. The answer is 9.

Q: [New complex question]
A: Let's think step by step."


In [ ]:
# Without Chain of Thought Prompting = Only Few Shot (No CoT)
prompt = f"""
Answer below question in consistent style, stick to the format
Teacher: Johan had 5 apples, he bought 4 more boxes with 5 apples each. How many apples does he have now?
Student: 25 apples

Teacher: Marry had 12 pens, she gave 10 to her friend. and bought 2 boxes of pen with 6 pen in each box. How many pens does she have now?
Student:
"""
response = generate_response(prompt)
print(response)

Student: 10 pens


In [20]:
# with Chain of Thought prompting = Few Shot CoT
prompt = f"""

Teacher: Johan had 5 apples, he bought 4 more boxes with 5 apples each. How many apples does he have now?
Student: Johan had 5 apples, 4 boxes with 5 apples each = 5 + 4*5 = 25 apples

Teacher: Marry had 12 pens, she gave 10 to her friend. and bought 2 boxes of pen with 6 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

To find out how many pens Marry has now, we can break it down step by step:

1. Marry started with 12 pens.
2. She gave away 10 pens, so now she has:  
   \( 12 - 10 = 2 \) pens.
3. She then bought 2 boxes of pens, with each box containing 6 pens. So, the total number of pens she bought is:  
   \( 2 \times 6 = 12 \) pens.
4. Now, we add the pens she has left to the pens she bought:  
   \( 2 + 12 = 14 \) pens.

So, Marry has 14 pens now.


In [21]:
# with Chain of Thought prompting = Zero shot CoT
prompt = f"""

Think step by step to answer below question, do not come to conclusion without following process.
Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

Let's break down the problem step by step:

1. **Initial Number of Pens**: Marry starts with 5 pens.

2. **Pens Given Away**: She gives 3 pens to her friend. 
   - To find out how many pens she has after giving away the pens, we subtract the number of pens given away from the initial number:
   \[
   5 - 3 = 2 \text{ pens}
   \]

3. **Buying New Pens**: Marry then buys 2 boxes of pens, with each box containing 3 pens.
   - First, we calculate the total number of pens in the boxes:
   \[
   2 \text{ boxes} \times 3 \text{ pens per box} = 6 \text{ pens}
   \]

4. **Total Pens After Buying**: Now, we add the number of pens she has after giving some away to the number of new pens she bought:
   \[
   2 \text{ pens} + 6 \text{ pens} = 8 \text{ pens}
   \]

5. **Final Count**: Therefore, after all the transactions, Marry has a total of 8 pens.

So, the final answer is that Marry has **8 pens** now.


In [22]:
# without Chain of thought

prompt = """
Based on the following eligibility criteria, determine which tier of the Premium Checking Account (Gold, Platinum, or None) the customer qualifies for.
Answer the question:
**Gold Tier Eligibility:**
- Maintain a minimum average daily balance of $5,000 OR
- Have at least $25,000 in combined linked accounts.
- Must also have at least 5 debit card transactions per month.

**Platinum Tier Eligibility:**
- Maintain a minimum average daily balance of $25,000 OR
- Have at least $100,000 in combined linked accounts.
- Must also have at least 10 debit card transactions per month.
- Additionally, must have a credit score of 700 or higher.

**Customer Profile:**
- Average daily checking balance: $8,000
- Combined linked accounts balance: $45,000
- Debit card transactions last month: 17
- Credit score: 720

Which tier does this customer qualify for?
"""
print(generate_response(prompt))

Based on the provided customer profile and the eligibility criteria for the Premium Checking Account:

1. **Gold Tier Eligibility:**
   - The customer maintains an average daily balance of $8,000, which exceeds the minimum requirement of $5,000.
   - The customer has $45,000 in combined linked accounts, which exceeds the minimum requirement of $25,000.
   - The customer has 17 debit card transactions, which exceeds the minimum requirement of 5 transactions.

   Therefore, the customer qualifies for the Gold Tier.

2. **Platinum Tier Eligibility:**
   - The customer maintains an average daily balance of $8,000, which does not meet the minimum requirement of $25,000.
   - The customer has $45,000 in combined linked accounts, which does not meet the minimum requirement of $100,000.
   - The customer has 17 debit card transactions, which exceeds the minimum requirement of 10 transactions.
   - The customer has a credit score of 720, which meets the requirement of 700 or higher.

   However

In [23]:
# Zero shot Chain of thought

prompt = """
Based on the following eligibility criteria, determine which tier of the Premium Checking Account (Gold, Platinum, or None) the customer qualifies for.
Answer the question:
**Gold Tier Eligibility:**
- Maintain a minimum average daily balance of $5,000 OR
- Have at least $25,000 in combined linked accounts.
- Must also have at least 5 debit card transactions per month.

**Platinum Tier Eligibility:**
- Maintain a minimum average daily balance of $25,000 OR
- Have at least $100,000 in combined linked accounts.
- Must also have at least 10 debit card transactions per month.
- Additionally, must have a credit score of 700 or higher.

**Customer Profile:**
- Average daily checking balance: $8,000
- Combined linked accounts balance: $15,000
- Debit card transactions last month: 7
- Credit score: 720

Which tier does this customer qualify for?
Lets analyze the customer profile step by step and try answer the question.
"""
print(generate_response(prompt))

Let's analyze the customer profile step by step based on the eligibility criteria for the Gold and Platinum tiers.

### Gold Tier Eligibility:
1. **Minimum average daily balance of $5,000**: 
   - The customer has an average daily checking balance of $8,000, which meets this requirement.
   
2. **Combined linked accounts balance of at least $25,000**: 
   - The customer has a combined linked accounts balance of $15,000, which does not meet this requirement.

3. **At least 5 debit card transactions per month**: 
   - The customer had 7 debit card transactions last month, which meets this requirement.

Since the customer meets the first and third requirements for the Gold Tier, they qualify for the Gold Tier.

### Platinum Tier Eligibility:
1. **Minimum average daily balance of $25,000**: 
   - The customer has an average daily checking balance of $8,000, which does not meet this requirement.

2. **Combined linked accounts balance of at least $100,000**: 
   - The customer has a combined

## Tree of Thoughts (ToT)

Tree of Thoughts (ToT) prompting is a framework that generalizes over chain-of-thought prompting and encourages exploration over thoughts that serve as intermediate steps for general problem-solving with language models.



**Description:** ToT is an extension of CoT where the model explores multiple reasoning paths (a tree structure) instead of a single linear chain of thought. It generates multiple intermediate thoughts and explores their consequences, allowing for backtracking and exploration of different possibilities before selecting the most promising path to the solution.

**Application Areas:** More complex and open-ended reasoning tasks than CoT, such as creative writing, strategic planning, and problem-solving that might require exploring multiple hypotheses.

**When to Use:**
- When the problem has multiple potential approaches or intermediate steps.
- When a single linear chain of thought might lead to local optima or dead ends.
- For tasks requiring more extensive exploration of possibilities.

**How it Works (conceptually):** ToT prompting breaks problems down into smaller parts, similar to CoT prompting, but goes further by combining this with the ability to explore multiple solution paths in parallel, forming a tree instead of a single chain. Each thought is generated or solved independently and passed to the next step, allowing the model to self-evaluate and decide whether to continue with that path or choose another.

Generate potential intermediate thoughts -> Evaluate their validity/promise -> Select the most promising ones -> Recursively generate further thoughts until a solution is reached. This typically requires more sophisticated prompting or external control mechanisms than basic CoT.


In [28]:

prompt = """
Imagine 5 different experts are answering this question.
They will brainstorm the answer step by step reasoning carefully and taking all facts into consideration
All experts will write down 1 step of their thinking,
then share it with the group.
They will each critique their response, and the all the responses of others
They will check their answer based on science and the laws of physics
They will keep going through steps until they reach their conclusion taking into account the thoughts of the other experts
If at any time they realise that there is a flaw in their logic they will backtrack to where that flaw occurred
If any expert realises they're wrong at any point then they acknowledges this and start another train of thought
Each expert will assign a likelihood of their current assertion being correct
Continue until the experts agree on the single most likely location
The question is.
1. Carlos is at the swimming pool.
2. He walks to the locker room, carrying a towel.
3. He puts his watch in the towel and carries the towel tightly to a lounger at the poolside.
4. At the lounger he opens and vigorously shakes the towel, then walks to the snack bar.
5. He leaves the towel at the snack bar, then walks to the diving board.
6. Later Carlos realises he has has lost his watch. Where is the single most likely location of the watch?
"""


ans=generate_response(prompt)

print(ans)

### Expert 1: The Physical Movement Expert
**Step 1:** Carlos starts at the swimming pool and walks to the locker room carrying a towel. The first thing to consider is that he puts his watch in the towel while still in the locker room. This means the watch is definitely in the towel at that point.

**Critique:** This step is solid as it establishes the initial location of the watch. However, I need to consider what happens next.

**Likelihood of correctness:** 90%

---

### Expert 2: The Towel Dynamics Expert
**Step 2:** After Carlos puts his watch in the towel, he carries it tightly to a lounger at the poolside. The act of carrying the towel tightly suggests that the watch is likely still secure within the towel during this movement.

**Critique:** This is a reasonable assumption, but I need to consider the possibility of the watch slipping out during the vigorous shaking at the lounger.

**Likelihood of correctness:** 85%

---

### Expert 3: The Shaking and Dropping Expert
**Step 3:*

In [24]:

# ToT Prompt for Loan Application Approval

prompt = """
Imagine 5 different loan officers are reviewing a customer's loan application.
They will brainstorm the decision step by step, reasoning carefully and taking all facts into consideration from the application.
Each loan officer will write down 1 key observation or analysis step from the application data,
then share it with the group.
They will each critique their response, and all the responses of others based on standard lending practices and risk assessment criteria.
They will check their analysis against established bank policies and economic indicators.
They will keep going through steps until they reach their conclusion on whether to approve or reject the loan, taking into account the observations and critiques of the other loan officers.
If at any time they realize there is a flaw in their analysis or a crucial piece of information is overlooked, they will backtrack to where that flaw occurred.
If any loan officer realizes their assessment is weak or incorrect at any point, they acknowledge this and start another train of thought, focusing on different aspects of the application.
Each loan officer will assign a likelihood (e.g., High Risk, Medium Risk, Low Risk) based on their current assertion about the application's overall risk profile.
Continue until the loan officers agree on the single most likely outcome (Approve or Reject) based on a consensus risk assessment.

The customer's loan application details are as follows:

Customer Name: [Customer Name]
Loan Amount Requested: [Amount]
Loan Type: [e.g., Personal Loan, Mortgage]
Annual Income: [Income]
Credit Score: [Score]
Employment Status: [e.g., Full-time, Self-employed]
Employment Duration: [Duration]
Existing Debt Obligations: [List or summary]
Assets: [List or summary]
Purpose of Loan: [Purpose]
Payment History: [Summary of past credit behavior]

Based on these details, where is the single most likely outcome of the loan application (Approve or Reject)?
"""

# Replace bracketed information with actual application details
customer_details_prompt = prompt.replace("[Customer Name]", "John Doe") \
                               .replace("[Amount]", "$50,000") \
                               .replace("[Loan Type]", "Personal Loan") \
                               .replace("[Income]", "$75,000") \
                               .replace("[Score]", "720") \
                               .replace("[Employment Status]", "Full-time") \
                               .replace("[Employment Duration]", "5 years") \
                               .replace("[Existing Debt Obligations]", "Credit card balance: $5,000, Auto loan payment: $400/month") \
                               .replace("[Assets]", "Savings account: $20,000, Car value: $15,000") \
                               .replace("[Purpose of Loan]", "Home renovation") \
                               .replace("[Summary of past credit behavior]", "No late payments in the last 3 years")

ans = generate_response(customer_details_prompt)

print(ans)


To analyze John Doe's loan application, the five loan officers will take turns sharing their observations and critiques based on the provided details. 

### Loan Officer 1: Initial Observation
**Observation:** John has a credit score of 720, which is considered good and indicates a history of responsible credit use. This is a positive factor for loan approval.

**Critique:** While the credit score is strong, we need to consider other factors such as his existing debt obligations and the purpose of the loan to fully assess risk.

**Risk Assessment:** Low Risk based on credit score alone.

---

### Loan Officer 2: Employment Status and Income
**Observation:** John has an annual income of $75,000, which is reasonable for a loan amount of $50,000. However, we need to confirm his employment status and duration to ensure job stability.

**Critique:** We need to clarify whether John is full-time or self-employed, as self-employment can introduce variability in income. Additionally, we should 

In [30]:
# ToT Prompt for Enterprise Banking Scenario: Supply Chain Finance Optimization

prompt = """
Imagine 5 different experts in Supply Chain Finance, Risk Management, Data Science, Operations, and Customer Relationship Management are collaborating to address the following challenge:

A large manufacturing company, a client of your bank, is experiencing significant working capital strain due to long payment terms with their suppliers (averaging 90 days)
and relatively short payment terms from their buyers (averaging 45 days).
This creates a cash flow gap that they currently bridge using expensive short-term loans from other institutions.

Your bank wants to propose a comprehensive Supply Chain Finance (SCF) solution that benefits both the manufacturing company (the buyer) and their key suppliers,
 increases the bank's revenue through new products, and strengthens the overall client relationship.
 The goal is not just to offer a standard factoring or reverse factoring product, but to design an integrated, optimized solution leveraging data and
 potentially incorporating risk mitigation strategies for the bank.

The experts will brainstorm the design of this SCF solution step by step, reasoning carefully and taking all relevant factors into consideration for the buyer,
their suppliers, and the bank.

All experts will write down 1 step of their thinking related to designing this optimized SCF solution, then share it with the group.
They will each critique their response, and all the responses of others.
They will consider the perspectives and constraints of the buyer (manufacturing company), their diverse set of suppliers (ranging from large corporations to small businesses), and the bank (risk appetite, operational capacity, revenue goals).
They will keep going through steps until they reach a well-defined and compelling SCF solution proposal, taking into account the thoughts of the other experts.
If at any time they realize that there is a flaw in their logic or a critical aspect is missed, they will backtrack to where that flaw occurred.
If any expert realizes they're wrong at any point, they acknowledge this and start another train of thought.
Each expert will assign a likelihood of their current proposed element of the solution being successful or viable.

Continue until the experts agree on the single most effective and viable integrated SCF solution proposal that addresses the working capital needs of the buyer and their suppliers while creating a valuable new revenue stream and strengthening the relationship for the bank.

The challenge is:
Design an optimized and integrated Supply Chain Finance solution for a large manufacturing client facing a working capital gap due to mismatched payment terms, considering the needs of their suppliers and the bank's objectives.
"""

ans = generate_response(prompt)
print(ans)


### Step 1: Supply Chain Finance Expert

**Proposal:** Implement a dynamic discounting program that allows the manufacturing company to offer early payment discounts to suppliers in exchange for faster payment. This program would be facilitated through a digital platform that automates the discount calculations and payment processes.

**Rationale:** By offering early payment discounts, the manufacturing company can reduce its working capital strain while providing suppliers with the option to receive payments sooner. This can help strengthen supplier relationships and improve the overall supply chain efficiency.

**Likelihood of Success:** 75%

---

### Step 2: Risk Management Expert

**Critique of Step 1:** The dynamic discounting program is a solid idea, but we need to consider the risk of suppliers not taking the discount due to their own cash flow issues. Additionally, we should assess the credit risk of suppliers to ensure that the bank is not exposed to defaults.

**New Proposal:

## Chain-of-Verification (CoVe) Prompting

**Description:** Chain-of-Verification (CoVe) is a prompting technique designed to improve the factual accuracy of Large Language Models (LLMs) by explicitly prompting the model to generate a draft response, then generate verification questions for its own response, answer those questions independently of the initial draft, and finally use the answers to revise and improve the original draft. This creates a self-correction mechanism.

**Application Areas:** Tasks requiring high factual accuracy and reduced hallucination, such as question answering, summarization of factual information, and generating reports or summaries based on provided text. It is particularly useful when dealing with complex or potentially ambiguous information where the model might otherwise confidently generate incorrect details.

**When to Use:**
- When factual accuracy is critical and errors could be harmful or misleading.
- To reduce hallucinations and improve the reliability of the model's output.
- When verifying information against external sources is not immediately feasible or desired within the prompting process itself.
- For generating summaries or answers that require synthesizing information from multiple potential points within the model's knowledge or context.

**How it Works:**
1. **Generate Initial Draft:** The model first generates a direct answer or response to the original prompt.
2. **Generate Verification Questions:** The model is then prompted to generate questions that would help verify the correctness of the statements made in the initial draft.
3. **Answer Verification Questions:** The model answers these verification questions *independently* of the initial draft, drawing from its internal knowledge or provided context. This step is crucial as it prevents the model from simply confirming its own potential errors.
4. **Revise Initial Draft:** The model compares the answers to the verification questions with the original draft and revises the draft to align with the verified information.

This iterative process helps the model identify inconsistencies or inaccuracies in its initial output and correct them before presenting the final response.

**Example (Conceptual):**

Prompt: "Explain the process of photosynthesis and its main outputs."

1.  **Initial Draft (Internal):** "Photosynthesis is how plants make food. They use sunlight, water, and carbon dioxide. The main output is oxygen and sugar." (Might contain slight inaccuracies or missing details).
2.  **Generate Verification Questions (Internal):**
    *   "What are the inputs of photosynthesis?"
    *   "What are the outputs of photosynthesis?"
    *   "What is the role of chlorophyll?"
    *   "Where does photosynthesis primarily occur in a plant?"
3.  **Answer Verification Questions (Internal, independently):**
    *   "Inputs are carbon dioxide, water, and light energy."
    *   "Outputs are glucose (a sugar) and oxygen."
    *   "Chlorophyll is a pigment that absorbs light energy."
    *   "It occurs mainly in chloroplasts within plant cells."
4.  **Revise Initial Draft (Internal, based on verified answers):** "Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods with the help of chlorophyll. During photosynthesis, plants take in carbon dioxide from the air, water from the soil, and light energy from the sun. Using the energy absorbed by chlorophyll in their chloroplasts, they convert these inputs into glucose, a sugar they use for energy and growth, and oxygen, which is released as a byproduct."

The final output presented to the user would be the revised draft. While the user doesn't see the intermediate steps, the process happens internally within the model's processing based on the CoVe prompt structure.


In [29]:
Question="Name some athletes who were born in United states"

prompt=f'''Here is the question: {Question}.

First, generate a response.

Then, create and answer verification questions based on this response related to original question to check for accuracy.
make sure the verification questions are related to original question {Question} and your answer.
Think it through and make sure you are extremely accurate based on the question asked.

After answering each verification question, consider these answers and revise the initial response to formulate a final, verified answer.
Ensure the final response reflects the accuracy and findings from the verification process.'''



ans=generate_response(prompt,temperature=0)

print(ans)


### Initial Response
Some athletes who were born in the United States include:
1. Michael Jordan (Basketball)
2. Serena Williams (Tennis)
3. Tom Brady (American Football)
4. Simone Biles (Gymnastics)
5. LeBron James (Basketball)

### Verification Questions
1. **Question:** Which sport does Michael Jordan play?
   - **Answer:** Basketball.
   
2. **Question:** Is Serena Williams known for her achievements in tennis?
   - **Answer:** Yes, she is a renowned tennis player.

3. **Question:** What sport is Tom Brady associated with?
   - **Answer:** American Football.

4. **Question:** What is Simone Biles famous for?
   - **Answer:** She is famous for her accomplishments in gymnastics.

5. **Question:** Is LeBron James also a basketball player?
   - **Answer:** Yes, he is a professional basketball player.

### Revision of Initial Response
After reviewing the verification questions and their answers, it is confirmed that all athletes listed are indeed born in the United States and are associ

In [30]:
prompt = """
In an enterprise banking context, a customer is requesting a complex derivative trade.
This involves intricate details regarding the underlying asset, notional value, tenor, strike price, option type (if applicable),
counterparty credit risk, regulatory compliance, and internal risk limits.

The task is to assess the viability and risks of this specific derivative trade request for the bank.

First, generate an initial assessment of the trade request, including potential benefits and immediate red flags based on standard derivative trading practices
and market knowledge.

Then, create and answer a series of verification questions specifically tailored to validate the critical details and associated risks mentioned in the initial assessment.
 Think through each question carefully, drawing upon knowledge of financial instruments, risk management frameworks, regulatory requirements (e.g., Dodd-Frank, EMIR),
 and internal banking policies.

Examples of potential verification questions:
- Is the counterparty's credit risk within acceptable limits for a trade of this size and tenor?
- Does this trade comply with all relevant regulatory requirements for derivative transactions?
- Does the pricing of this derivative accurately reflect current market conditions and volatility?
- How does this trade impact the bank's overall exposure to the underlying asset?
- Are the operational processes and systems in place to handle the settlement and ongoing management of this specific derivative type?
- Has the client undergone the necessary suitability and appropriateness checks for complex products like this?

After answering each verification question independently, consider these answers and revise the initial assessment to formulate a final,
 verified decision on whether to proceed with the trade, including any necessary conditions or risk mitigation steps.
 Ensure the final response is a comprehensive analysis reflecting the accuracy and findings from the verification process and aligns with the bank's risk appetite and policies.

The customer's derivative trade request details are:
[Provide specific details of the trade request here, e.g.,
Underlying Asset: EUR/USD currency pair
Notional Value: $100,000,000
Tenor: 2 years
Trade Type: FX Forward
Client: [Client Name], Credit Rating: [Rating]
Purpose: Hedging foreign exchange exposure]
"""

# Replace bracketed information with actual trade request details
trade_details_prompt = prompt.replace("[Provide specific details of the trade request here...]",
                                       """Underlying Asset: Interest Rate Swap (Fixed-to-Floating USD)
Notional Value: $250,000,000
Tenor: 5 years
Trade Type: Receive Fixed, Pay Floating
Client: [Client Name], Credit Rating: AA-
Purpose: Hedge against rising interest rates on existing floating-rate debt""")

ans = generate_response(trade_details_prompt)
print(ans)


### Initial Assessment of the Trade Request

**Trade Details:**
- **Underlying Asset:** EUR/USD currency pair
- **Notional Value:** $100,000,000
- **Tenor:** 2 years
- **Trade Type:** FX Forward
- **Client:** [Client Name]
- **Credit Rating:** [Rating]
- **Purpose:** Hedging foreign exchange exposure

**Potential Benefits:**
1. **Hedging:** The primary purpose of the trade is to hedge against foreign exchange exposure, which can stabilize cash flows and protect against adverse currency movements.
2. **Market Positioning:** If the client has a strong view on the EUR/USD pair, this trade could allow them to lock in favorable rates, potentially benefiting both the client and the bank through increased business.
3. **Revenue Generation:** The bank may earn fees or spreads from facilitating this trade, contributing positively to its revenue.

**Immediate Red Flags:**
1. **Credit Risk:** The size of the notional value ($100,000,000) raises concerns about the counterparty's creditworthiness. 

## ReAct Prompting
**Description:** ReAct (Reasoning and Acting) prompting is a technique that combines reasoning trace generation with task-specific actions. It encourages the model to interleave thinking steps (Reasoning) with taking actions (Acting) through tools or interacting with an environment. The model observes the outcomes of its actions and uses this information to refine its subsequent reasoning and actions. This allows the model to perform tasks that require interacting with external sources of information or performing physical (or simulated physical) actions.

**Application Areas:** Tasks that require external knowledge lookup, interacting with APIs or tools, complex decision-making, and environments where actions have observable consequences. Examples include answering questions that require searching the web, using a calculator for complex math, interacting with a simulated game environment, or completing tasks that involve multiple external steps.

**When to Use:**
- When the task requires up-to-date information not present in the model's training data.
- When the task involves complex calculations or operations that the model is not inherently good at performing reliably.
- When the task requires interacting with external systems or data sources.
- To enable the model to break down a complex task into smaller, executable steps based on external feedback.

**How it Works (Simulated in a Prompt):** In a prompt, ReAct is simulated by guiding the model to explicitly state its "Thought" process (reasoning about the next step or action) and its "Action" (the action it would take or the tool it would use), followed by an "Observation" (the simulated result of the action). The model then uses the Observation to inform its next Thought and Action.


In [32]:
prompt = """
You are a helpful assistant that can answer questions by first thinking step-by-step about the question, then deciding on an action (like searching or using a tool),
observing the result of the action, and then repeating the process until you can formulate a final answer.

Your process should follow this format:

Thought: Think step-by-step about the question and what information you need to answer it.
Action: [Describe the action you would take or tool you would use]
Observation: [Simulate the result of the action]
... (Repeat Thought, Action, Observation until you have enough information)
Thought: Now that I have the necessary information, I can formulate the final answer.
Final Answer: [Provide the final answer]

Here is the question: What is the current population of Tokyo?

"""

ans = generate_response(prompt)
print(ans)


Thought: To answer the question about the current population of Tokyo, I need to find the most recent population statistics. Tokyo is a large metropolitan area, and its population can vary depending on whether I'm looking for the population of the city proper or the greater metropolitan area. I should clarify which population figure is being requested and check for the latest data available.

Action: I will search for the latest population statistics for Tokyo, focusing on both the city proper and the metropolitan area.

Observation: I find that as of 2023, the population of Tokyo's 23 special wards (the city proper) is approximately 14 million people, while the Greater Tokyo Area, which includes surrounding prefectures, has a population of about 37 million people.

Thought: Now that I have the population figures for both the city proper and the greater metropolitan area, I can determine which figure is more relevant based on common usage. Typically, when people refer to the population

In [33]:

prompt = """
You are an expert financial analyst assistant. You need to determine if a company's stock is overvalued based on its current P/E ratio compared to the industry average.

Follow the ReAct process:

Thought: Think step-by-step about how to assess if the stock is overvalued based on the P/E ratio.
Action: [Describe the action to find the company's current stock price and EPS to calculate its P/E ratio]
Observation: [Simulate the result, e.g., "Company X Stock Price: $150, EPS: $5.00"]
Thought: Calculate the company's P/E ratio.
Action: [Describe the calculation: Stock Price / EPS]
Observation: [Simulate the result, e.g., "Company X P/E Ratio: 30x"]
Thought: Find the industry average P/E ratio for comparison.
Action: [Describe the action to find the industry average P/E ratio for the relevant sector (e.g., Technology)]
Observation: [Simulate the result, e.g., "Industry Average P/E Ratio (Technology): 25x"]
Thought: Compare the company's P/E ratio to the industry average and determine if it is overvalued.
Action: [Describe the comparison and decision process]
Observation: [Simulate the result of the comparison]
Thought: Formulate the final assessment.
Final Answer: [Provide the final assessment based on the P/E comparison]

Here is the company: Company X (Technology Sector)
"""

ans = generate_response(prompt)
print(ans)

**Thought:** To assess if Company X's stock is overvalued, I will first need to find its current stock price and earnings per share (EPS) to calculate its price-to-earnings (P/E) ratio. 

**Action:** I will look up Company X's current stock price and its EPS. For this example, let's assume I find that Company X's stock price is $150 and its EPS is $5.00.

**Observation:** Company X Stock Price: $150, EPS: $5.00

**Thought:** Now that I have the stock price and EPS, I can calculate Company X's P/E ratio.

**Action:** The P/E ratio is calculated by dividing the stock price by the EPS. Therefore, the calculation will be: 

\[ \text{P/E Ratio} = \frac{\text{Stock Price}}{\text{EPS}} = \frac{150}{5} \]

**Observation:** Company X P/E Ratio: 30x

**Thought:** Next, I need to find the industry average P/E ratio for the technology sector to compare it with Company X's P/E ratio.

**Action:** I will research the current industry average P/E ratio for the technology sector. For this example, let

## Conversational App

In [37]:
# creating a function to get outcome
def generate_response(messages,model='gpt-4o-mini'):
  messages2 = [{"role":"system","content":"You are a helpful assistant"},]
  messages2.append({"role":"user","content":messages})
  response = client.chat.completions.create(
      model = model,
      messages = messages2,
      temperature=0.5
  )
  return response.choices[0].message.content


In [38]:
messages = "System: You are a helpful assistant"
while True:
    uinput = input("You: ")
    print("You: ",uinput)
    if uinput.lower() in ['q','quit','exit']:
        break
    #messages.append({"role":"user","content":uinput})

    messages = messages + "User: "+uinput+"\n"
    response = generate_response(messages=messages)
    print(f"Bot: {response}")
    messages = messages + "Assistant: " + response + "\n"
    #messages.append({"role":"assistant","content":response})


You:  hi
Bot: Hello! How can I assist you today?
You:  i live in delhi
Bot: That's great! Delhi is a vibrant city with a rich history and culture. How can I help you today? Are you looking for information about places to visit, local events, or something else?
You:  q


In [35]:
messages = [{"role":"system","content":"You are a helpful assistant"},]
while True:
    uinput = input("You: ")
    print("You: ",uinput)
    if uinput.lower() in ['q','quit','exit']:
        break
    messages.append({"role":"user","content":uinput})
    response = generate_response(messages=messages)
    print(f"Bot: {response}")
    messages.append({"role":"assistant","content":response})


You:  hi
Bot: Hello! How can I assist you today?
You:  I live in Gurgaon
Bot: That's great! Gurgaon, now known as Gurugram, is a bustling city with a lot to offer. Are you looking for information on something specific, such as places to visit, dining options, or local events?
You:  what is ai?
Bot: AI, or artificial intelligence, refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. AI systems can perform tasks that typically require human intelligence, such as understanding natural language, recognizing patterns, solving problems, and making decisions.

There are several types of AI, including:

1. **Narrow AI**: Also known as weak AI, this type is designed to perform a specific task, such as voice recognition or image classification. Most AI applications today fall into this category.

2. **General AI**: Also known as strong AI, this type would have the ability to understand, learn, and apply intelligence across a wide range of

In [36]:
messages

[{'role': 'system', 'content': 'You are a helpful assistant'},
 {'role': 'user', 'content': 'hi'},
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'},
 {'role': 'user', 'content': 'I live in Gurgaon'},
 {'role': 'assistant',
  'content': "That's great! Gurgaon, now known as Gurugram, is a bustling city with a lot to offer. Are you looking for information on something specific, such as places to visit, dining options, or local events?"},
 {'role': 'user', 'content': 'what is ai?'},
 {'role': 'assistant',
  'content': 'AI, or artificial intelligence, refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. AI systems can perform tasks that typically require human intelligence, such as understanding natural language, recognizing patterns, solving problems, and making decisions.\n\nThere are several types of AI, including:\n\n1. **Narrow AI**: Also known as weak AI, this type is designed to perform a specific task, 

## Thank You